In [ ]:
import torch
from torch.utils.data import Dataset
import os

In [69]:
dir(os)

['DirEntry',
 'F_OK',
 'GenericAlias',
 'Mapping',
 'MutableMapping',
 'O_APPEND',
 'O_BINARY',
 'O_CREAT',
 'O_EXCL',
 'O_NOINHERIT',
 'O_RANDOM',
 'O_RDONLY',
 'O_RDWR',
 'O_SEQUENTIAL',
 'O_SHORT_LIVED',
 'O_TEMPORARY',
 'O_TEXT',
 'O_TRUNC',
 'O_WRONLY',
 'P_DETACH',
 'P_NOWAIT',
 'P_NOWAITO',
 'P_OVERLAY',
 'P_WAIT',
 'PathLike',
 'R_OK',
 'SEEK_CUR',
 'SEEK_END',
 'SEEK_SET',
 'TMP_MAX',
 'W_OK',
 'X_OK',
 '_AddedDllDirectory',
 '_Environ',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_check_methods',
 '_execvpe',
 '_exists',
 '_exit',
 '_fspath',
 '_get_exports_list',
 '_walk',
 '_wrap_close',
 'abc',
 'abort',
 'access',
 'add_dll_directory',
 'altsep',
 'chdir',
 'chmod',
 'close',
 'closerange',
 'cpu_count',
 'curdir',
 'defpath',
 'device_encoding',
 'devnull',
 'dup',
 'dup2',
 'environ',
 'error',
 'execl',
 'execle',
 'execlp',
 'execlpe',
 'execv',
 'execve',
 'execvp',
 'execvpe',
 'exts

In [70]:
from PIL import Image 
class Mydata(Dataset):
    def __init__(self,root_path,label_path):
        self.root_path=root_path
        self.label_path=label_path
        self.img_path=os.path.join(self.root_path,self.label_path)
        self.img_list=os.listdir(self.img_path)
    def __getitem__(self, idx):
        img_name=self.img_list[idx]
        img_item_path=os.path.join(self.img_path,img_name)
        img=Image.open(img_item_path)
        label=self.label_path
        return img,label 
    def __len__(self):
        return len(self.img_list)

In [71]:
root_path="E:\\DL\\hymenoptera_data\\hymenoptera_data\\train"
ants=Mydata(root_path,'ants')

In [72]:
img,_=ants[10]

In [73]:
bees=Mydata(root_path,'bees')
bee_img,_=bees[101]

In [74]:
totalData=ants+bees

In [75]:
from torchvision.transforms import ToTensor
img_tensor=ToTensor()
img=img_tensor(img)

In [76]:
type(img)

torch.Tensor

In [77]:
from torchvision import transforms
trans_norm=transforms.Normalize([0.5,0.5,0.5],[0.5,.5,0.5])
norm_img=trans_norm(img)

In [78]:
from torch.utils.tensorboard import SummaryWriter

writer=SummaryWriter('log')

In [79]:
for i in range(100):
    writer.add_scalar('y=4x',4*i,i)

In [80]:
writer.add_image('ants',img,global_step=1)

In [81]:
writer.add_image('ants',norm_img,global_step=2)

In [82]:
orginal_img,_=ants[11]
orginal_img.show()
orginal_img.size

(500, 375)

In [ ]:
Resizer=transforms.Resize((400,300))
resize_img=Resizer(orginal_img)
# resize_img.show()

In [84]:
trans_compose=transforms.Compose([transforms.Resize((400,300)),transforms.ToTensor(),transforms.Normalize([0.5,.5,.5],[.5,.5,.5])])

img_compose=trans_compose(bees[101][0])

writer.add_image('compose image',img_compose)

In [85]:
Crop=transforms.RandomCrop([200,200],pad_if_needed=True,padding_mode='edge')
crop_img=Crop(orginal_img)

tmp=img_tensor(crop_img)
tmp=trans_norm(tmp)
tmp.shape

torch.Size([3, 200, 200])

In [87]:
cropCompose=transforms.Compose([Crop,img_tensor])
i=0
for k,_ in ants:
    writer.add_image('crop',cropCompose(k),i)
    i+=1

In [93]:
tmp=trans_norm(cropCompose(ants[11][0]))
tmp.shape

torch.Size([3, 200, 200])

In [95]:
compose=transforms.Compose([cropCompose,trans_norm])

In [96]:
compose(ants[11][0])

tensor([[[ 0.4196,  0.3882,  0.4039,  ...,  0.2235,  0.2078,  0.2314],
         [ 0.3961,  0.3961,  0.3882,  ...,  0.2157,  0.2000,  0.2235],
         [ 0.3804,  0.3882,  0.3804,  ...,  0.2157,  0.2078,  0.2549],
         ...,
         [-0.3725, -0.3804, -0.3961,  ..., -0.2314, -0.2471, -0.2157],
         [-0.4510, -0.4275, -0.4353,  ..., -0.1922, -0.2157, -0.2000],
         [-0.4510, -0.4980, -0.4745,  ..., -0.1608, -0.1765, -0.1686]],

        [[ 0.4196,  0.3882,  0.4039,  ...,  0.1843,  0.1686,  0.2000],
         [ 0.3882,  0.3961,  0.3882,  ...,  0.2000,  0.1686,  0.1922],
         [ 0.3804,  0.3882,  0.3804,  ...,  0.2078,  0.2000,  0.2314],
         ...,
         [-0.4039, -0.4118, -0.4039,  ..., -0.2471, -0.2627, -0.2314],
         [-0.5137, -0.4902, -0.4980,  ..., -0.2000, -0.2235, -0.2078],
         [-0.5373, -0.5843, -0.5608,  ..., -0.1765, -0.1843, -0.1765]],

        [[ 0.4196,  0.3882,  0.4039,  ...,  0.2314,  0.2000,  0.2078],
         [ 0.3569,  0.3804,  0.4039,  ...,  0

In [97]:
cropCompose1=transforms.Compose([Crop,img_tensor])
cropCompose2=transforms.Compose([Crop,img_tensor,trans_norm])

In [98]:
cropCompose1(ants[11][0])
cropCompose2(ants[11][0])


tensor([[[-0.7255, -0.9922, -0.6863,  ...,  0.4510,  0.4431,  0.4431],
         [-0.7647, -0.9373, -1.0000,  ...,  0.4510,  0.4431,  0.4431],
         [-0.7647, -0.8275, -1.0000,  ...,  0.4353,  0.4353,  0.4353],
         ...,
         [-0.1216, -0.0196, -0.0510,  ...,  0.0196,  0.0353,  0.0353],
         [ 0.0118,  0.0118, -0.0588,  ...,  0.0588,  0.0745,  0.0431],
         [-0.0039,  0.0667, -0.0353,  ...,  0.0745,  0.0510,  0.0510]],

        [[-0.7725, -1.0000, -0.6863,  ...,  0.4510,  0.4431,  0.4431],
         [-0.8118, -0.9765, -0.9843,  ...,  0.4510,  0.4431,  0.4431],
         [-0.8353, -0.8745, -0.9843,  ...,  0.4510,  0.4431,  0.4431],
         ...,
         [-0.0510,  0.0431,  0.0118,  ...,  0.0039,  0.0039,  0.0039],
         [ 0.0745,  0.0745, -0.0196,  ...,  0.0118,  0.0196,  0.0118],
         [ 0.0353,  0.0980, -0.0039,  ...,  0.0039, -0.0275, -0.0275]],

        [[-0.7412, -0.9843, -0.7020,  ...,  0.4510,  0.4431,  0.4588],
         [-0.7804, -0.9451, -1.0000,  ...,  0

In [ ]:
cropCompose2=transforms.Compose([Crop,img_tensor,trans_norm])
cropCompose2(ants[11][0])
for k,_ in ants:
    cropCompose2(k)
    i+=1

(768, 512)
(500, 333)
(500, 282)
(500, 335)
(500, 348)
(500, 321)
(500, 500)
(500, 333)
(500, 375)
(500, 375)
(500, 375)
(500, 375)
(441, 500)
(500, 375)
(500, 400)
(500, 375)
(500, 375)
(500, 374)
(343, 496)
(500, 333)
(500, 415)
(500, 330)
(500, 333)
(479, 359)
(500, 462)
(500, 314)
(500, 375)
(500, 361)
(500, 423)
(500, 400)
(500, 333)
(500, 375)
(500, 333)
(500, 375)
(500, 236)
(500, 375)
(500, 375)
(500, 333)
(500, 409)
(500, 409)
(500, 434)
(500, 334)
(500, 500)
(500, 500)
(500, 333)
(500, 375)
(500, 325)
(500, 357)
(357, 500)
(500, 375)
(500, 375)
(375, 500)
(500, 333)
(500, 375)
(500, 375)
(500, 375)
(399, 500)
(488, 500)
(500, 358)
(500, 384)
(500, 333)
(500, 333)
(500, 356)
(500, 333)
(500, 375)
(500, 407)
(500, 333)
(500, 374)
(500, 407)
(500, 444)
(500, 374)
(500, 333)
(333, 500)
(333, 500)
(500, 397)
(500, 313)
(500, 333)
(500, 333)
(500, 339)
(500, 400)
(500, 375)
(500, 332)
(500, 333)
(417, 500)
(500, 375)
(500, 239)
(500, 333)
(500, 402)
(500, 432)
(500, 375)
(500, 375)

RuntimeError: output with shape [1, 200, 200] doesn't match the broadcast shape [3, 200, 200]